In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
import numpy as np

In [26]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')     

In [27]:
data = pd.read_csv("../mydata/cleandata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)
foods = sorted(list(set(data['product'])))

In [28]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "carrefour"
prices = []
prod = []
sup = []
links = []
names = []
newprice = []
frame = {}
for f in foods:
    for i in range(2,4):
        url = f"https://www.carrefour.es/?q={f}&page={i}"
        driver.get(url)
        time.sleep(0.3)
        try: #cookies
            driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
            time.sleep(0.7)
        except:
            pass
        try:
            for o in range(1,50): #price, scroll, product, supermarket, link and name
                    prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
                    driver.execute_script("window.scrollTo(0, window.scrollY + 300)")
                    prod.append(f)
                    sup.append('carrefour')
                    links.append(url)
                    names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)
        except:
            pass

frame['price'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['link'] = links
frame['name'] = names

scrap = pd.DataFrame(frame)
for i, row in scrap.iterrows():
    if row['price'] == '':
        newprice.append(np.nan)
    else:
        newprice.append(row['price'])
scrap['price'] = newprice
scrap.dropna(how='any',inplace=True)
scrap['price'] = [float((i.replace(",",".").replace("€","").strip())) for i in scrap['price']]
scrap['name'] = [(i.replace("%","por ciento").strip()) for i in scrap['name']]


if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
now = str(datetime.now())[:19].replace(":","_")
scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv', index=False)
scrap.to_csv(f'../mydata/scraps_{supermarket}/scrap.csv', index=False)

/tmp/ipykernel_643/3252668423.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_643/3252668423.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_643/3252668423.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
/tmp/ipykernel_643/3252668423.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)


In [22]:
scrap.isna().sum()

price          0
product        0
supermarket    0
link           0
name           0
dtype: int64